# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-services Model provides an API to all model services and facilitates all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)
model

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.author = :author_1 AND model.laboratory = :laboratory_1 AND model.facility = :facility_1 AND model.beampath = :beampath_1 AND model.description = :description_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 9, 28, 19, 32, 58),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [ ]:
# create a project
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)
scheduling_service.create_project("test")

You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model
Replace `source_path` with the path to your release tarball below:

In [6]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.42/my_model-0.0.42.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.42/my_model-0.0.42.tar.gz saved to /tmp/lume-services/sources/my_model-0.0.42.tar.gz
INFO:lume_services.environment.solver:Version 0.0.42 of my_model already installed.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 AND deployment.version = :version_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow.flow_id, flow.flow_name, flow.project_name, flow.deployment_id 
FROM flow 
WHERE flow.deployment_id = :deployment_id_1
INFO:lume_services.models.model:Loading deployment 2
INFO:lume_services.models.model:Loading deploym

## Run the Prefect workflow directly

In [7]:
flow_run = model.deployment.flow.prefect_flow.run(**{
                        "input1": 1, 
                        "input2": 2, 
                        "filename": "test_file.txt", 
                        "filesystem_identifier":"local"
    }
)

[2022-09-28 13:55:05-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'my-model'


INFO:prefect.FlowRunner:Beginning Flow run for 'my-model'


[2022-09-28 13:55:05-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Starting task run...


INFO:prefect.TaskRunner:Task 'configure_lume_services': Starting task run...
INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


[2022-09-28 13:55:05-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Finished task run for task with final state: 'Success'


...

INFO:prefect.TaskRunner:Task 'save_file': Starting task run...


[2022-09-28 13:55:06-0700] INFO - prefect.TaskRunner | Task 'save_file': Finished task run for task with final state: 'Success'


INFO:prefect.TaskRunner:Task 'save_file': Finished task run for task with final state: 'Success'


[2022-09-28 13:55:06-0700] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


INFO:prefect.FlowRunner:Flow run SUCCESS: all reference tasks succeeded


## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [8]:
task=model.deployment.flow.prefect_flow.get_tasks(name="save_db_result")[0]

In [9]:
flow_run.result[task].load_result().result

{'result_type_string': 'lume_services.results.generic:Result',
 'query': {'unique_hash': '9e39546869490f71b0683ef7a1a16f8e'}}

In [10]:
model.run(
    parameters={
        "input1": 1, 
        "input2": 2, 
        "filename": "/lume-services/data/test_file.txt", 
        "filesystem_identifier":"mounted"}
)

# Get results:

In [11]:
results = model.get_results(collection="generic")
results

[Result(model_type='generic', id='6334a263d8f9468b948635d6', flow_id='f14310fa-633f-4e66-9b48-36a24bd65787', inputs={'input1': 1.0, 'input2': 2.0}, outputs={'output1': array([[1.8205216 , 1.87573707, 1.31580732, ..., 1.47352911, 1.54806234,
         1.50754862],
        [1.19381883, 1.49009244, 1.13622791, ..., 1.92061284, 1.51974611,
         1.10668552],
        [1.27128172, 1.55534337, 1.65552294, ..., 1.6463503 , 1.49166613,
         1.62441884],
        ...,
        [1.9686497 , 1.85915591, 1.07250613, ..., 1.75061297, 1.31430971,
         1.1109343 ],
        [1.22910587, 1.05297709, 1.40537252, ..., 1.81010662, 1.39598882,
         1.5787685 ],
        [1.15467431, 1.72622978, 1.76217663, ..., 1.81354314, 1.47558494,
         1.94026862]]), 'output2': 1.0, 'output3': 2.0}, date_modified=datetime.datetime(2022, 9, 28, 19, 37, 4, 838000), unique_on=['inputs', 'outputs', 'flow_id'], unique_hash='ef83125284df7dfeaad201b193bee95e', result_type_string='lume_services.results.generic:Re

In [12]:
results_df = model.get_results_df(collection="generic")
results_df

,model_type,flow_id,input1,input2,output1,output2,output3,date_modified,unique_hash,result_type_string
0,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.820521598226592, 1.875737065714508, 1.3158...",1.0,2.0,2022-09-28 19:37:04.838,ef83125284df7dfeaad201b193bee95e,lume_services.results.generic:Result
1,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.8594920966103436, 1.9702209676716067, 1.47...",1.0,2.0,2022-09-28 19:41:06.989,f4686ede779b73bfe1423e1c20f3ecc9,lume_services.results.generic:Result
2,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.7838111640117922, 1.3434367993756404, 1.66...",1.0,2.0,2022-09-28 19:43:22.873,1b123234753e1b730e16038484a9c16d,lume_services.results.generic:Result
3,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.9657229574316597, 1.9373403094863368, 1.34...",1.0,2.0,2022-09-28 19:44:19.275,31617d7af1975c0c9e7023d59dbdf1fc,lume_services.results.generic:Result
4,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.0249553697721896, 1.795705610400978, 1.168...",1.0,2.0,2022-09-28 19:45:04.945,d7675b858df4f7b92ea5bc75f1bec212,lume_services.results.generic:Result
5,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.9931810327610155, 1.9855442434420671, 1.40...",1.0,2.0,2022-09-28 19:49:02.182,c1f8505027930a36a58444349742cbe5,lume_services.results.generic:Result
6,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.589720734266254, 1.1951683055177797, 1.533...",1.0,2.0,2022-09-28 19:53:28.744,11bb70000277ac024a6f09b73298cced,lume_services.results.generic:Result
7,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.1636674576639654, 1.441257712865944, 1.677...",1.0,2.0,2022-09-28 19:54:35.063,f8d388a4d7401b9312ff14a462cb975d,lume_services.results.generic:Result
8,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.9692685443468778, 1.7361192123149523, 1.57...",1.0,2.0,2022-09-28 19:55:10.782,d019e294cd4ebffd423dd9fe87e1d2e6,lume_services.results.generic:Result
9,generic,f14310fa-633f-4e66-9b48-36a24bd65787,1.0,2.0,"[[1.2943132791561838, 1.7766031230507902, 1.91...",1.0,2.0,2022-09-28 19:59:09.176,0199cb2d1126cd828eefa913a5ed351d,lume_services.results.generic:Result


## Load model using model id
Once your model has been registered, you can use the `Model` api object to interact with your model without running the above registration steps. Let's load a new model object using the model_id we registered above.

In [13]:
from importlib import reload 
import lume_services
reload(lume_services.models)
from lume_services.models import Model

model_id = model.metadata.model_id
loaded_model = Model(model_id=model_id)

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


In [14]:
loaded_model.metadata

Model(                     model_id=1,                     created=datetime.datetime(2022, 9, 28, 19, 32, 58),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 )

## Load existing model object
Loading a model using the load_deployment method without passing a deployment_id will load the latest deployment registered for the model.

In [15]:
loaded_model.load_deployment()
loaded_model.deployment

INFO:lume_services.models.model:Loading latest deployment.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 ORDER BY deployment.deploy_date DESC
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow.flow_id, flow.flow_name, flow.project_name, flow.deployment_id 
FROM flow 
WHERE flow.deployment_id = :deployment_id_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT project.project_name, project.description 
FROM project 
WHERE project.project_name = :project_name_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow_of_flows._id, flow_of_flows.parent_flow_id, flow_of_flows.flow_id, flow_of_flows.position 
FROM flow_of_flows 
WHERE flow_of_fl

Deployment(metadata=Deployment(                 deployment_id=2,                 model_id=1,                 version='0.0.42',                 deploy_date=datetime.datetime(2022, 9, 28, 19, 35, 42)),                 asset_dir=None,                 source='https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.42/my_model-0.0.42.tar.gz',                 sha256='0f05676f30df43a94370de8b3aba01189164724b32823e69bdf4f06d894b0bef',                 image='registry.hub.docker.com/jgarrahan/my-model:v0.0.42',                 is_live=True                 package_import_name='my_model'                 ), project=Project(metadata=Project(                 project_name='test',                 description='my_description',                 )), flow=Flow(name='my-model', flow_id='f14310fa-633f-4e66-9b48-36a24bd65787', project_name='test', prefect_flow=None, parameters=None, mapped_parameters=None, task_slugs=None, labels=['lume-services'], image='registry.hub.docker.com/jgarrahan/my-mode

In [16]:
results = loaded_model.run_and_return(
    collection="generic",
    parameters={
                    "input1": 1.0, 
                    "input2": 2.0, 
                    "filename": "/lume-services/data/test_file.txt", 
                    "filesystem_identifier":"local"
    },
    task_name="save_db_result" # Want to get the result from the save_db_result task
)
res

INFO:lume_services.services.scheduling.backends.server:Creating Prefect flow run for f14310fa-633f-4e66-9b48-36a24bd65787 with parameters {'input1': 1.0, 'input2': 2.0, 'filename': '/lume-services/data/test_file.txt', 'filesystem_identifier': 'local'} and run_config {"labels": ["lume-services"], "env": null, "image": "registry.hub.docker.com/jgarrahan/my-model:v0.0.42", "host_config": null, "ports": null}
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 9, 28, 20, 55, 11, 937835, tzinfo=Timezone('+00:00')), level=20, message='Entered state <Scheduled>: Flow run scheduled.')
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 9, 28, 20, 55, 12, 367533, tzinfo=Timezone('+00:00')), level=20, message='Entered state <Submitted>: Submitted for execution')
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 9, 28, 20, 55, 13, 658299, tzinfo=Timezone('+00:00')), level=20, mes

INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 9, 28, 20, 55, 18, 237926, tzinfo=Timezone('+00:00')), level=20, message='Flow run SUCCESS: all reference tasks succeeded')
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 9, 28, 20, 55, 18, 262987, tzinfo=Timezone('+00:00')), level=20, message='Entered state <Success>: All reference tasks succeeded.')


{'result_type_string': 'lume_services.results.generic:Result', 'query': {'unique_hash': '16943181c206f4484ca9d0f8659dd803'}}


[Result(model_type='generic', id='6334b4b61d7c780e74f3b3d4', flow_id='f14310fa-633f-4e66-9b48-36a24bd65787', inputs={'input1': 1.0, 'input2': 2.0}, outputs={'output1': array([[1.26067391, 1.8450996 , 1.19084657, ..., 1.836936  , 1.89644075,
         1.71541681],
        [1.70550379, 1.96994645, 1.89248388, ..., 1.15608557, 1.18246536,
         1.63169308],
        [1.46134652, 1.19440154, 1.60908337, ..., 1.30561834, 1.13128327,
         1.07777527],
        ...,
        [1.97880652, 1.44029066, 1.80755682, ..., 1.636144  , 1.81926731,
         1.11353966],
        [1.61223866, 1.90658222, 1.49289798, ..., 1.31623637, 1.0211756 ,
         1.81705073],
        [1.91228904, 1.97362938, 1.77886776, ..., 1.09737881, 1.39736749,
         1.43171737]]), 'output2': 1.0, 'output3': 2.0}, date_modified=datetime.datetime(2022, 9, 28, 20, 55, 15, 711000), unique_on=['inputs', 'outputs', 'flow_id'], unique_hash='16943181c206f4484ca9d0f8659dd803', result_type_string='lume_services.results.generic:R